# Snowflakes

Algorithmically generate snowflakes.

In [ ]:
import drawsvg as dw
import numpy as np
import ipywidgets as widgets
from IPython.display import display

In [ ]:
def draw_snowflake():
    img_size: int = 800
    radius: float = 0.9 * img_size / 2
    spine_radius: float = 0.2 * radius
    spine_theta: float = np.pi / 6
    rng = np.random.default_rng()
    n_spines: int = rng.integers(4, 20)
    spine_r = np.sort(rng.uniform(0, 1, n_spines))
    # spine_length = np.sort(rng.uniform(0, 1, n_spines))
    spine_length = rng.uniform(0, 1, n_spines)
    # print(spine_r)
    d = dw.Drawing(img_size, img_size, origin='center')
    d.append(dw.Rectangle(-img_size, -img_size, 2 * img_size, 2 * img_size, fill='#1A1A2E'))
    for idx in range(6):
        line = dw.Line(0, 0, radius * np.sin(idx * np.pi / 3), radius * np.cos(idx * np.pi / 3), stroke='white')
        d.append(line)
        for spine_idx in range(n_spines):
            r = spine_r[spine_idx]
            length = spine_radius * spine_length[spine_idx]
            x0 = radius * r * np.sin(idx * np.pi / 3)
            y0 = radius * r * np.cos(idx * np.pi / 3)
            x1 = x0 + length * np.sin(idx * np.pi / 3 + spine_theta)
            y1 = y0 + length * np.cos(idx * np.pi / 3 + spine_theta)
            x2 = x0 + length * np.sin(idx * np.pi / 3 - spine_theta)
            y2 = y0 + length * np.cos(idx * np.pi / 3 - spine_theta)
            line = dw.Line(x0, y0, x1, y1, stroke='white')
            d.append(line)
            line = dw.Line(x0, y0, x2, y2, stroke='white')
            d.append(line)
    return d

draw_snowflake()

# Recursive Snowflake Generation

Use a recursive algorithm to generate a snowflake using generation rules similar to L-systems.

We will define the snowflake with a series of parameters:

- depth: The number of recursive iterations to perform.
- branch_angle: The angle between branches.
- branch_points: A list of branch points for each iteration, indicating where along the branch to draw a new branch.
- branch_lengths: A list of branch lengths for each iteration.


In [ ]:
def draw_branch(
    d: dw.Drawing,
    x0: float,
    y0: float,
    line_length: float,
    current_depth: int,
    max_depth: int,
    n_branch_points_array: np.ndarray,
    branch_radius_matrix: np.ndarray,
    branch_length_matrix: np.ndarray,
    current_branch_angle: float,
    branch_angle: float,
    attenuation_factor: float,
    stroke_color: str
):
    """Recursively draw a branch and its sub-branches."""
    # Draw the current branch
    x1: float = x0 + line_length * np.cos(current_branch_angle)
    y1: float = y0 + line_length * np.sin(current_branch_angle)
    line = dw.Line(x0, y0, x1, y1, stroke=stroke_color)
    d.append(line)
    
    # Base case: stop recursion
    if current_depth >= max_depth:
        return
    
    # Calculate branch points along the current branch
    n_branch_points: int = n_branch_points_array[current_depth]
    branch_radius_array: np.ndarray = branch_radius_matrix[current_depth][0:n_branch_points]
    branch_length_array: np.ndarray = branch_length_matrix[current_depth][0:n_branch_points]
    
    # Position along the branch
    bx = x0 + (x1 - x0) * branch_radius_array
    by = y0 + (y1 - y0) * branch_radius_array
    for branch_idx in range(n_branch_points):
        for sign in [1, -1]:
            draw_branch(
                d=d,
                x0=bx[branch_idx],
                y0=by[branch_idx],
                line_length=branch_length_array[branch_idx] * line_length * attenuation_factor,
                current_depth=current_depth + 1,
                max_depth=max_depth,
                n_branch_points_array=n_branch_points_array,
                branch_radius_matrix=branch_radius_matrix,
                branch_length_matrix=branch_length_matrix,
                current_branch_angle=current_branch_angle + sign * branch_angle,
                branch_angle=branch_angle,
                attenuation_factor=attenuation_factor,
                stroke_color=stroke_color
            )


def draw_recursive_snowflake(
    max_depth: int = 3,
    n_poles: int = 6,
    background: str = '#1A1A2E',
    stroke_color: str = 'white',
    seed: int = None
):
    """Draw a recursive snowflake with customizable parameters.
    
    Args:
        max_depth: Number of recursive iterations (0-5 recommended)
        n_poles: Number of poles for the snowflake. Default is 6 (hexagonal symmetry)
        background: Background color (hex or named color)
        stroke_color: Line color (hex or named color)
        seed: Random seed for reproducibility
    """
    img_size: int = 800
    rng = np.random.default_rng() # seed
    attenuation_factor: float = 0.5
    branch_angle: float = 2 * np.pi / n_poles
    n_branch_points_array: np.ndarray = rng.integers(1, 8, max_depth)
    branch_radius_matrix: np.ndarray = rng.uniform(0, 1, (max_depth, n_branch_points_array.max()))
    branch_length_matrix: np.ndarray = rng.uniform(0, 1, (max_depth, n_branch_points_array.max()))
    
    d = dw.Drawing(img_size, img_size, origin='center')
    d.append(dw.Rectangle(-img_size, -img_size, 2 * img_size, 2 * img_size, fill=background))
    
    
    line_length: float = 0.9 * img_size / 2
    for idx in range(n_poles):
        draw_branch(
            d=d,
            x0=0,
            y0=0,
            line_length=line_length,
            current_depth=0,
            max_depth=max_depth,
            n_branch_points_array=n_branch_points_array,
            branch_radius_matrix=branch_radius_matrix,
            branch_length_matrix=branch_length_matrix,
            current_branch_angle=idx * branch_angle,
            branch_angle=branch_angle,
            attenuation_factor=attenuation_factor,
            stroke_color=stroke_color
        )
    
    return d


draw_recursive_snowflake(
    max_depth=3,
)

# Interactive Snowflake Generator

Use the widgets below to customize your snowflake in real-time!

In [ ]:
# Create output widget to display the snowflake
output = widgets.Output()

# Create interactive widgets
max_depth_slider = widgets.IntSlider(
    value=3,
    min=0,
    max=5,
    step=1,
    description='Depth:',
    continuous_update=False
)

n_poles_slider = widgets.IntSlider(
    value=6,
    min=3,
    max=12,
    step=1,
    description='Number of Poles:',
    continuous_update=False
)

background_picker = widgets.Dropdown(
    options=['#1A1A2E (Dark Blue)', '#FFFFFF (White)', '#000000 (Black)', '#E8F4F8 (Ice Blue)'],
    value='#1A1A2E (Dark Blue)',
    description='Background:'
)

stroke_picker = widgets.Dropdown(
    options=['white', 'black', '#00D9FF (Cyan)', '#FFD700 (Gold)'],
    value='white',
    description='Stroke Color:'
)

# seed_input = widgets.IntText(
#     value=42,
#     description='Random Seed:'
# )

generate_button = widgets.Button(
    description='Generate Snowflake',
    button_style='primary',
    icon='snowflake'
)

def on_generate_click(b):
    """Generate and display snowflake when button is clicked."""
    with output:
        output.clear_output(wait=True)
        
        # Extract color codes from dropdown values
        bg_color = background_picker.value.split(' ')[0]
        stroke_color = stroke_picker.value.split(' ')[0]
        
        # Convert angle from degrees to radians
        angle_rad = angle_slider.value * np.pi / 180
        
        # Generate snowflake
        snowflake = draw_recursive_snowflake(
            max_depth=max_depth_slider.value,
            n_poles=n_poles_slider.value,
            # branch_angle=angle_rad,
            # img_size=size_slider.value,
            background=bg_color,
            stroke_color=stroke_color,
            # seed=seed_input.value
        )
        
        display(snowflake)

generate_button.on_click(on_generate_click)

# Layout the widgets
controls = widgets.VBox([
    widgets.HTML('<h3>Snowflake Parameters</h3>'),
    max_depth_slider,
    n_poles_slider,
    background_picker,
    stroke_picker,
    # seed_input,
    generate_button
])

# Display the interface
display(widgets.HBox([controls, output]))

# Generate initial snowflake
on_generate_click(None)